In [24]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib as plt
import os

In [25]:
shots_df = pd.read_csv("Resources/shot_logs.csv")

In [26]:
shots_df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


In [27]:
df = shots_df.drop(columns=["GAME_ID", "LOCATION", "W",
               "FINAL_MARGIN", "PERIOD", 
               "CLOSEST_DEFENDER_PLAYER_ID",
               "FGM","PTS","player_id"])

In [28]:
df2 = df.rename(columns={"player_name":"player", "SHOT_NUMBER": "shot_number", 
                   "GAME_CLOCK": "game_clock", "SHOT_CLOCK": "shot_clock",
                  "DRIBBLES": "dribbles", "TOUCH_TIME": "touch_time",
                  "SHOT_DIST": "shot_distance", "PTS_TYPE": "shot_type",
                  "SHOT_RESULT": "shot_result", "CLOSEST_DEFENDER": "closest_defender",
                  "CLOSE_DEF_DIST": "closest_defender_distance"})
new_df = df2[['player', 'shot_number', 'closest_defender',
                'closest_defender_distance','game_clock','shot_clock',
                'dribbles', 'touch_time', 'shot_distance', 'shot_type',
                'shot_result']]

new_df.head()

,player,shot_number,closest_defender,closest_defender_distance,game_clock,shot_clock,dribbles,touch_time,shot_distance,shot_type,shot_result
0,brian roberts,1,"Anderson, Alan",1.3,1:09,10.8,2,1.9,7.7,2,made
1,brian roberts,2,"Bogdanovic, Bojan",6.1,0:14,3.4,0,0.8,28.2,3,missed
2,brian roberts,3,"Bogdanovic, Bojan",0.9,0:00,NaN,3,2.7,10.1,2,missed
3,brian roberts,4,"Brown, Markel",3.4,11:47,10.3,2,1.9,17.2,2,missed
4,brian roberts,5,"Young, Thaddeus",1.1,10:34,10.9,2,2.7,3.7,2,missed


In [29]:
new_df['player']= new_df['player'].str.title()
new_df.head()

C:\Users\jseymour\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,player,shot_number,closest_defender,closest_defender_distance,game_clock,shot_clock,dribbles,touch_time,shot_distance,shot_type,shot_result
0,Brian Roberts,1,"Anderson, Alan",1.3,1:09,10.8,2,1.9,7.7,2,made
1,Brian Roberts,2,"Bogdanovic, Bojan",6.1,0:14,3.4,0,0.8,28.2,3,missed
2,Brian Roberts,3,"Bogdanovic, Bojan",0.9,0:00,NaN,3,2.7,10.1,2,missed
3,Brian Roberts,4,"Brown, Markel",3.4,11:47,10.3,2,1.9,17.2,2,missed
4,Brian Roberts,5,"Young, Thaddeus",1.1,10:34,10.9,2,2.7,3.7,2,missed


In [30]:
new_df.head()

,player,shot_number,closest_defender,closest_defender_distance,game_clock,shot_clock,dribbles,touch_time,shot_distance,shot_type,shot_result
0,Brian Roberts,1,"Anderson, Alan",1.3,1:09,10.8,2,1.9,7.7,2,made
1,Brian Roberts,2,"Bogdanovic, Bojan",6.1,0:14,3.4,0,0.8,28.2,3,missed
2,Brian Roberts,3,"Bogdanovic, Bojan",0.9,0:00,NaN,3,2.7,10.1,2,missed
3,Brian Roberts,4,"Brown, Markel",3.4,11:47,10.3,2,1.9,17.2,2,missed
4,Brian Roberts,5,"Young, Thaddeus",1.1,10:34,10.9,2,2.7,3.7,2,missed


In [31]:
def reverse_name(name):
    try:
        split_name = name.split(",")
        last_name = split_name[0]
        first_name = split_name[1]
        return first_name + " " + last_name
    except:
        print(split_name)

new_df['closest_defender'] = new_df['closest_defender'].apply(reverse_name)


['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
['Nene']
[

C:\Users\jseymour\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [32]:
new_df.head(50)

,player,shot_number,closest_defender,closest_defender_distance,game_clock,shot_clock,dribbles,touch_time,shot_distance,shot_type,shot_result
0,Brian Roberts,1,Alan Anderson,1.3,1:09,10.8,2,1.9,7.7,2,made
1,Brian Roberts,2,Bojan Bogdanovic,6.1,0:14,3.4,0,0.8,28.2,3,missed
2,Brian Roberts,3,Bojan Bogdanovic,0.9,0:00,NaN,3,2.7,10.1,2,missed
3,Brian Roberts,4,Markel Brown,3.4,11:47,10.3,2,1.9,17.2,2,missed
4,Brian Roberts,5,Thaddeus Young,1.1,10:34,10.9,2,2.7,3.7,2,missed
5,Brian Roberts,6,Deron Williams,2.6,8:15,9.1,2,4.4,18.4,2,missed
6,Brian Roberts,7,Jarrett Jack,6.1,10:15,14.5,11,9.0,20.7,2,missed
7,Brian Roberts,8,Mason Plumlee,2.1,8:00,3.4,3,2.5,3.5,2,made
8,Brian Roberts,9,Darius Morris,7.3,5:14,12.4,0,0.8,24.6,3,missed
9,Brian Roberts,1,Wayne Ellington,19.8,11:32,17.4,0,1.1,22.4,3,missed


In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 11 columns):
player                       128069 non-null object
shot_number                  128069 non-null int64
closest_defender             127680 non-null object
closest_defender_distance    128069 non-null float64
game_clock                   128069 non-null object
shot_clock                   122502 non-null float64
dribbles                     128069 non-null int64
touch_time                   128069 non-null float64
shot_distance                128069 non-null float64
shot_type                    128069 non-null int64
shot_result                  128069 non-null object
dtypes: float64(4), int64(3), object(4)
memory usage: 10.7+ MB


In [34]:
new_df['player'] = new_df['player'].astype('str')

C:\Users\jseymour\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 11 columns):
player                       128069 non-null object
shot_number                  128069 non-null int64
closest_defender             127680 non-null object
closest_defender_distance    128069 non-null float64
game_clock                   128069 non-null object
shot_clock                   122502 non-null float64
dribbles                     128069 non-null int64
touch_time                   128069 non-null float64
shot_distance                128069 non-null float64
shot_type                    128069 non-null int64
shot_result                  128069 non-null object
dtypes: float64(4), int64(3), object(4)
memory usage: 10.7+ MB


In [36]:
new_df.to_csv('shot_stats.csv')

In [37]:
#create the connection to the db
rds_connection_string = "postgres:postgres@localhost:5432/NBA_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [38]:
engine.table_names()

['players', 'shot_stats']

In [41]:
new_df.to_sql(name='shot_stats', con=engine, if_exists='append', index=False)

In [42]:
pd.read_sql_query('select * from shot_stats', con=engine).head()

,player,shot_number,closest_defender,closest_defender_distance,game_clock,shot_clock,dribbles,touch_time,shot_distance,shot_type,shot_result
0,Brian Roberts,1,Alan Anderson,1,01:09:00,11.0,2,2,8,2,made
1,Brian Roberts,2,Bojan Bogdanovic,6,00:14:00,3.0,0,1,28,3,missed
2,Brian Roberts,3,Bojan Bogdanovic,1,00:00:00,NaN,3,3,10,2,missed
3,Brian Roberts,4,Markel Brown,3,11:47:00,10.0,2,2,17,2,missed
4,Brian Roberts,5,Thaddeus Young,1,10:34:00,11.0,2,3,4,2,missed


In [43]:
pd.read_sql_query('select player from shot_stats', con=engine).head()

,player
0,Brian Roberts
1,Brian Roberts
2,Brian Roberts
3,Brian Roberts
4,Brian Roberts
